In [1]:
import pandas as pd
import numpy as np

## 分组计算

分组计算三步曲：拆分 -> 应用 -> 合并
* 拆分：根据什么进行分组？
* 应用：每个分组进行什么样的计算？
* 合并：把每个分组的计算结果合并起来。

![groupby](imgs/groupby.png)

In [2]:
df = pd.DataFrame({'key1': ['a', 'a', 'b', 'b', 'a'],
                  'key2': ['one', 'two', 'one', 'two', 'one'],
                  'data1': np.random.randint(1, 10, 5),
                  'data2': np.random.randint(1, 10, 5)})
df

,key1,key2,data1,data2
0,a,one,2,7
1,a,two,9,6
2,b,one,1,1
3,b,two,9,5
4,a,one,6,2


## 对Series进行分组
通过索引对齐关联起来

In [3]:
grouped = df['data1'].groupby(df['key1']).sum()
grouped

key1
a    17
b    10
Name: data1, dtype: int64

In [4]:
df['data1'].groupby([df['key1'],df['key2']]).mean()

key1  key2
a     one     4
      two     9
b     one     1
      two     9
Name: data1, dtype: int64

## 对DataFrame进行分组
通过索引对齐关联起来

In [5]:
df.groupby(by='key1').mean()


,data1,data2
key1,,
a,5.666667,5.0
b,5.000000,3.0


In [6]:
means = df.groupby(by=['key1','key2']).mean()
means

data1  data2
key1 key2              
a    one     4.0    4.5
     two     9.0    6.0
b    one     1.0    1.0
     two     9.0    5.0

## 每个分组的元素个数

In [7]:
df.groupby(['key1','key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

## 对分组进行迭代

In [8]:
for name, group in df.groupby('key1'):
    print(name)
    print(group)

a
  key1 key2  data1  data2
0    a  one      2      7
1    a  two      9      6
4    a  one      6      2
b
  key1 key2  data1  data2
2    b  one      1      1
3    b  two      9      5


## 转换为字典

In [9]:
d = dict(list(df.groupby('key1')))
d

{'a':   key1 key2  data1  data2
 0    a  one      2      7
 1    a  two      9      6
 4    a  one      6      2,
 'b':   key1 key2  data1  data2
 2    b  one      1      1
 3    b  two      9      5}

In [10]:
d['a']

,key1,key2,data1,data2
0,a,one,2,7
1,a,two,9,6
4,a,one,6,2


## 按列分组

In [11]:
df.dtypes

key1     object
key2     object
data1     int64
data2     int64
dtype: object

In [12]:
grouped = df.groupby(df.dtypes,axis=1).sum()
grouped
# list(grouped)

,int64,object
0,9,aone
1,15,atwo
2,2,bone
3,14,btwo
4,8,aone


# 通过字典进行分组

In [13]:
df = pd.DataFrame(np.random.randint(1, 10, (5, 5)),
                  columns=['a', 'b', 'c', 'd', 'e'],
                  index=['Alice', 'Bob', 'Candy', 'Dark', 'Emily'])
df



,a,b,c,d,e
Alice,1,6,4,6,3
Bob,8,6,4,1,4
Candy,1,3,3,1,7
Dark,8,2,5,8,4
Emily,3,1,7,4,4


In [14]:
df.iloc[1,1:3] = np.NaN
df

,a,b,c,d,e
Alice,1,6.0,4.0,6,3
Bob,8,NaN,NaN,1,4
Candy,1,3.0,3.0,1,7
Dark,8,2.0,5.0,8,4
Emily,3,1.0,7.0,4,4


In [15]:
mapping = {'a':'red','b':'red','c': 'blue', 'd': 'orange', 'e': 'blue'}
grouped = df.groupby(mapping,axis=1)

In [16]:
grouped.sum()

,blue,orange,red
Alice,7.0,6.0,7.0
Bob,4.0,1.0,8.0
Candy,10.0,1.0,4.0
Dark,9.0,8.0,10.0
Emily,11.0,4.0,4.0


In [17]:
grouped.count()

,blue,orange,red
Alice,2,1,2
Bob,1,1,1
Candy,2,1,2
Dark,2,1,2
Emily,2,1,2


In [18]:
grouped.size()

blue      2
orange    1
red       2
dtype: int64

## 通过函数来分组
当函数作为分组依据时，数据表里的每个索引（可以是行索引，也可以是列索引）都会调用一次函数，函数的返回值作为分组的索引，即相同的返回值分在同一组。

In [19]:
# 传入索引值
def _dummpy_groupy(idx):
    print(idx)
    return idx
df.groupby(_dummpy_groupy)

Alice
Bob
Candy
Dark
Emily


In [20]:
df.groupby(_dummpy_groupy,axis=1)

a
b
c
d
e


In [21]:
df

,a,b,c,d,e
Alice,1,6.0,4.0,6,3
Bob,8,NaN,NaN,1,4
Candy,1,3.0,3.0,1,7
Dark,8,2.0,5.0,8,4
Emily,3,1.0,7.0,4,4


In [22]:
grouped = df.groupby(len)

In [23]:
grouped.sum()

,a,b,c,d,e
3,8,0.0,0.0,1,4
4,8,2.0,5.0,8,4
5,5,10.0,14.0,11,14


In [24]:
grouped.count()

,a,b,c,d,e
3,1,0,0,1,1
4,1,1,1,1,1
5,3,3,3,3,3


## 多级索引数据根据索引级别来分组

In [25]:
columns = pd.MultiIndex.from_arrays([['China', 'USA', 'China', 'USA', 'China'],['A', 'A', 'B', 'C', 'B']],names=['country', 'index'])
df = pd.DataFrame(np.random.randint(1,10,(5,5)), columns=columns)
df

country,China,USA,China,USA,China
index,A,A,B,C,B
0,1,5,8,8,6
1,6,6,7,4,4
2,5,5,7,5,7
3,5,8,8,2,5
4,1,2,4,8,9


In [26]:
df.groupby(level='country',axis=1).count()

country,China,USA
0,3,2
1,3,2
2,3,2
3,3,2
4,3,2


In [27]:
df.groupby(level='country',axis=1).sum()

country,China,USA
0,15,13
1,17,10
2,19,10
3,18,10
4,14,10


In [28]:
df.groupby(level='index',axis=1).count()

index,A,B,C
0,2,2,1
1,2,2,1
2,2,2,1
3,2,2,1
4,2,2,1


## 数据聚合
分组运算，先根据一定规则拆分后的数据，然后对数据进行聚合运算，如前面见到的 `mean()`, `sum()` 等就是聚合的例子。聚合时，拆分后的第一个索引指定的数据都会依次传给聚合函数进行运算。最后再把运算结果合并起来，生成最终结果。
聚合函数除了内置的 `sum()`, `min()`, `max()`, `mean()` 等等之外，还可以自定义聚合函数。自定义聚合函数时，使用 `agg()` 或 `aggregate()` 函数。

## 内置聚合函数

In [29]:
df = pd.DataFrame({'key1': ['a', 'a', 'b', 'b', 'a'],
                  'key2': ['one', 'two', 'one', 'two', 'one'],
                  'data1': np.random.randint(1, 10, 5),
                  'data2': np.random.randint(1, 10, 5)})
df

,key1,key2,data1,data2
0,a,one,5,6
1,a,two,9,9
2,b,one,9,7
3,b,two,8,1
4,a,one,3,4


In [30]:
df['data1'].groupby(df['key1']).sum()

key1
a    17
b    17
Name: data1, dtype: int64

### 自定义聚合函数

In [31]:
def peak_verbose(s):
    # print(type(s))
    print(s)
    # print(s.values)
    return s.max() - s.min()

def peak(s):
    print(type(s))
    return s.max() - s.min()

In [32]:
grouped = df.groupby('key1')
grouped

In [33]:
grouped.agg(peak_verbose)

0    one
1    two
4    one
Name: key2, dtype: object
0    one
1    two
4    one
Name: a, dtype: object
0    5
1    9
4    3
Name: data1, dtype: int64
2    9
3    8
Name: data1, dtype: int64
0    6
1    9
4    4
Name: data2, dtype: int64
2    7
3    1
Name: data2, dtype: int64


,data1,data2
key1,,
a,6,5
b,1,6


In [34]:
grouped.agg(peak)

<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>


,data1,data2
key1,,
a,6,5
b,1,6


In [35]:
df.max()

key1       b
key2     two
data1      9
data2      9
dtype: object

## 应用多个聚合函数

In [36]:
# grouped.agg(['mean','std',peak])
grouped['data1','data2'].agg(['mean','std',peak])

<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


data1                    data2               
          mean       std peak      mean       std peak
key1                                                  
a     5.666667  3.055050    6  6.333333  2.516611    5
b     8.500000  0.707107    1  4.000000  4.242641    6

In [37]:
grouped['data1'].agg([('agerage','mean'),('max-range',peak)])

<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>


,agerage,max-range
key1,,
a,5.666667,6
b,8.500000,1


## 给不同的列应用不同的聚合函数
使用 dict 作为参数来实现

In [38]:
d = {'data1': ['mean', peak, 'max', 'min'],
     'data2': 'sum'}
grouped.agg(d)

<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>


data1              data2
          mean peak max min   sum
key1                             
a     5.666667    6   9   3    19
b     8.500000    1   9   8     8

## 重置索引

In [39]:
grouped.agg(d).reset_index()

<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>


key1     data1              data2
            mean peak max min   sum
0    a  5.666667    6   9   3    19
1    b  8.500000    1   9   8     8

In [40]:
df.groupby('key1',as_index=False).agg(d)

<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>


key1     data1              data2
            mean peak max min   sum
0    a  5.666667    6   9   3    19
1    b  8.500000    1   9   8     8

## 分组运算和转换
groupby 是特殊的分组运算。更一般的分组运算包括 “拆分 - 应用 - 合并”。这里介绍 `transform()` 和 `apply()` 来实现分组运算。

In [41]:
df = pd.DataFrame({'key1': ['a', 'a', 'b', 'b', 'a'],
                  'key2': ['one', 'two', 'one', 'two', 'one'],
                  'data1': np.random.randint(1, 10, 5),
                  'data2': np.random.randint(1, 10, 5)})
df



,key1,key2,data1,data2
0,a,one,8,8
1,a,two,5,2
2,b,one,4,9
3,b,two,8,7
4,a,one,4,9


In [42]:
k1_mean = df.groupby('key1').mean().add_prefix('abc_')
k1_mean

,abc_data1,abc_data2
key1,,
a,5.666667,6.333333
b,6.000000,8.000000


In [43]:
pd.merge(df,k1_mean,left_on='key1',right_index=True)

,key1,key2,data1,data2,abc_data1,abc_data2
0,a,one,8,8,5.666667,6.333333
1,a,two,5,2,5.666667,6.333333
4,a,one,4,9,5.666667,6.333333
2,b,one,4,9,6.000000,8.000000
3,b,two,8,7,6.000000,8.000000


In [44]:
df

,key1,key2,data1,data2
0,a,one,8,8
1,a,two,5,2
2,b,one,4,9
3,b,two,8,7
4,a,one,4,9


In [45]:
# 使用 transform 简化处理
k1_mean = df.groupby('key1').transform(np.mean).add_prefix('mean___')
k1_mean

,mean___data1,mean___data2
0,5.666667,6.333333
1,5.666667,6.333333
2,6.000000,8.000000
3,6.000000,8.000000
4,5.666667,6.333333


In [46]:
df[k1_mean.columns] = k1_mean
df

,key1,key2,data1,data2,mean___data1,mean___data2
0,a,one,8,8,5.666667,6.333333
1,a,two,5,2,5.666667,6.333333
2,b,one,4,9,6.000000,8.000000
3,b,two,8,7,6.000000,8.000000
4,a,one,4,9,5.666667,6.333333


## 距平化
与平均值的差异值

In [47]:
df = pd.DataFrame(np.random.randint(1, 10, (5, 5)),
                  columns=['a', 'b', 'c', 'd', 'e'],
                  index=['Alice', 'Bob', 'Candy', 'Dark', 'Emily'])
df


,a,b,c,d,e
Alice,1,7,3,2,5
Bob,1,7,5,1,4
Candy,9,1,4,5,9
Dark,6,7,5,2,8
Emily,8,4,4,7,7


In [48]:
def demean(s):
    return s - s.mean()
key = ['one', 'one', 'two', 'one', 'two']
demeaned = df.groupby(key).transform(demean)
demeaned

,a,b,c,d,e
Alice,-1.666667,0.0,-1.333333,0.333333,-0.666667
Bob,-1.666667,0.0,0.666667,-0.666667,-1.666667
Candy,0.500000,-1.5,0.000000,-1.000000,1.000000
Dark,3.333333,0.0,0.666667,0.333333,2.333333
Emily,-0.500000,1.5,0.000000,1.000000,-1.000000


In [49]:
demeaned.groupby(key).mean()

,a,b,c,d,e
one,1.480297e-16,0.0,2.960595e-16,-7.401487e-17,-2.960595e-16
two,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.000000e+00


## apply函数
我们介绍过 DataFrame 的 apply 函数是逐行或逐列来处理数据。GroupBy 的 apply 函数对每个分组进行计算。


In [50]:
df = pd.DataFrame({'key1': ['a', 'a', 'b', 'b', 'a', 'a', 'a', 'b', 'b', 'a'],
                  'key2': ['one', 'two', 'one', 'two', 'one', 'one', 'two', 'one', 'two', 'one'],
                  'data1': np.random.randint(1, 10, 10),
                  'data2': np.random.randint(1, 10, 10)})
df


,key1,key2,data1,data2
0,a,one,9,6
1,a,two,2,1
2,b,one,5,3
3,b,two,2,8
4,a,one,9,3
5,a,one,7,8
6,a,two,6,1
7,b,one,9,9
8,b,two,3,2
9,a,one,9,8


In [51]:
# 根据 column 排序，输出其最大的 n 行数据
def top(df, n=2, column='data1'):
    return df.sort_values(by=column, ascending=False)[:n]

# top(df, n=5)

In [52]:
df.groupby('key1').apply(top)

key1 key2  data1  data2
key1                          
a    0    a  one      9      6
     4    a  one      9      3
b    7    b  one      9      9
     2    b  one      5      3

In [53]:
df.groupby('key1').apply(top,n=3,column='data2')


key1 key2  data1  data2
key1                          
a    5    a  one      7      8
     9    a  one      9      8
     0    a  one      9      6
b    7    b  one      9      9
     3    b  two      2      8
     2    b  one      5      3

In [57]:
# 禁用分组键
df.groupby('key1',group_keys=False).apply(top)


,key1,key2,data1,data2
0,a,one,9,6
4,a,one,9,3
7,b,one,9,9
2,b,one,5,3


## apply 应用示例：用不同的分组平均值填充空缺数据

In [58]:
states = ['Ohio', 'New York', 'Vermont', 'Florida',
          'Oregon', 'Nevada', 'California', 'Idaho']
group_key = ['East'] * 4 + ['West'] * 4
data = pd.Series(np.random.randn(8), index=states)
data[['Vermont', 'Nevada', 'Idaho']] = np.nan
data

Ohio          0.190571
New York      0.729647
Vermont            NaN
Florida       0.358883
Oregon        1.188723
Nevada             NaN
California   -0.073364
Idaho              NaN
dtype: float64

In [59]:
data.groupby(group_key).mean()

East    0.426367
West    0.557680
dtype: float64

In [60]:
fill_mean = lambda g: g.fillna(g.mean())
data.groupby(group_key).apply(fill_mean)

Ohio          0.190571
New York      0.729647
Vermont       0.426367
Florida       0.358883
Oregon        1.188723
Nevada        0.557680
California   -0.073364
Idaho         0.557680
dtype: float64